# Capítulo 2 - Estruturas de dados do pandas

## Seção 2 - Criando seus próprios dados

Para criar uma série no pandas podemos passar uma lista. Toda a série será classificada como de um tipo. Caso passemos uma lista com tipos misturados, o dtype será um object.

In [1]:
import pandas as pd

s = pd.Series(['banana', 42])
print(s)

0    banana
1        42
dtype: object


Na exibição da lista o número da linha é exibido à esquerda. Isso na verdade é o índice da série. Podemos dar nomes para os índices.

In [2]:
s = pd.Series(['banana', 42], ['Fruta', 'Resposta para todas as perguntas'])
print(s)

Fruta                               banana
Resposta para todas as perguntas        42
dtype: object


Podemos passar outros tipos de containers para a criação de uma Series. No caso de um dict, as chaves comporão o index e os valores a série.

In [3]:
dict_banana_42 = {'Fruta': 'banana', 'Resposta': 42}
print(pd.Series(dict_banana_42))

Fruta       banana
Resposta        42
dtype: object


Se passarmos também um index, tudo será reindexado de acordo com a nova ordem dos índices. Por exemplo, na linha abaixo mudamos a 'Fruta' para a segunda posição e criamos uma nova entrada com 'nova_chave'. A entrada referente a 'Fruta' permanece como banana. A 'nova_chave' fica com NaN pois não existia originalmente.

In [4]:
print(pd.Series(dict_banana_42, index=['nova_chave', 'Fruta']))

nova_chave       NaN
Fruta         banana
dtype: object


Podemos criar um dataframe a partir de um dicionário de listas

In [5]:
cientistas = pd.DataFrame({
    'Nome': ['Einstein', 'Rosaline Franklin'],
    'Ocupação': ['Físico', 'Química'],
    'Idade': [37, 61]
})
print(cientistas)

                Nome Ocupação  Idade
0           Einstein   Físico     37
1  Rosaline Franklin  Química     61


Podemos definir a ordem a partir do parâmetro columns. Se for o caso, podemos também nomear as linhas (index)

In [6]:
cientistas = pd.DataFrame({
    'Ocupação': ['Físico', 'Química'],
    'Idade': [37, 61]},
    index=['Einstein', 'Rosaline Franklin'],
    columns=['Idade', 'Ocupação'])
print(cientistas)

                   Idade Ocupação
Einstein              37   Físico
Rosaline Franklin     61  Química


## Seção 3 - Series

O acesso a uma linha do dataframe retorna um objeto do tipo Series:

In [7]:
primeira_linha = cientistas.loc['Einstein']
print(type(primeira_linha))
print(primeira_linha)

<class 'pandas.core.series.Series'>
Idade           37
Ocupação    Físico
Name: Einstein, dtype: object


A série tem um atributo index que retorna o nome dos índices e um atributo values que retorna os valores. Existe também o método keys() que nada mais é que um alias para o index.

In [8]:
print(primeira_linha.index)
print(primeira_linha.keys())

print(primeira_linha.values)

Index(['Idade', 'Ocupação'], dtype='object')
Index(['Idade', 'Ocupação'], dtype='object')
[37 'Físico']


Uma série é muito semelhante a um ndarray. Muitos métodos de ndarray se aplicam também a uma série. Quando uma série é numérica, podemos aplicar diretamente alguns métodos (min, max, median, std etc):

In [9]:
idades = cientistas['Idade']
print(idades.min())
print(idades.mean())
print(idades.max())

37
49.0
61


A seleção de dados feita com loc/iloc presume que sabemos exatamente o que queremos acessar (o índice, por exemplo). Obviamente para encontrar o índice fazemos algum pré-processamento em busca de alguma informação. Por exemplo, se quisermos encontrar todos os cientistas que tem mais de 50 anos o jeito tradicional é fazer um loop no vetor de idade encontrando os índices referentes a idades maiores do que 50 e, aí sim, usar os índices para acessar os objetos.

Com pandas podemos acessar os objetos de uma série ou dataframe através de booleanos que representam se queremos que o objeto seja incluído ou não no retorno. E esses booleanos podem ser calculados através de operações sobre as séries de um dataframe. Para exemplificar, vamos aplicar essa ideia no exemplo de separar os cientistas com mais de 50 anos.

Primeiro vamos ler um arquivo com mais dados do que o nosso exemplo inicial com apenas dois registros e mostrar as idades de todos eles:

In [10]:
cientistas = pd.read_csv('../data/scientists.csv')
idades = cientistas['Age']
print(idades)
idades.describe()


0    37
1    61
2    90
3    66
4    56
5    45
6    41
7    77
Name: Age, dtype: int64


count     8.000000
mean     59.125000
std      18.325918
min      37.000000
25%      44.000000
50%      58.500000
75%      68.750000
max      90.000000
Name: Age, dtype: float64

Podemos gerar um vetor de booleanos (na verdade é um objeto do tipo pandas.core.series.Series, mas podemos pensar como se fosse um vetor de booleanos) mostrando quais idades tem mais do que 50 anos:

In [11]:
mais_50_anos = idades > 50
print(mais_50_anos)
print(type(mais_50_anos))

0    False
1     True
2     True
3     True
4     True
5    False
6    False
7     True
Name: Age, dtype: bool
<class 'pandas.core.series.Series'>


E aí podemos usar esse série de booleanos para acessar os dados da série ou mesmo do dataframe:

In [12]:
print(idades[mais_50_anos])
print(cientistas[mais_50_anos])

1    61
2    90
3    66
4    56
7    77
Name: Age, dtype: int64
                   Name        Born        Died  Age     Occupation
1        William Gosset  1876-06-13  1937-10-16   61   Statistician
2  Florence Nightingale  1820-05-12  1910-08-13   90          Nurse
3           Marie Curie  1867-11-07  1934-07-04   66        Chemist
4         Rachel Carson  1907-05-27  1964-04-14   56      Biologist
7          Johann Gauss  1777-04-30  1855-02-23   77  Mathematician


## Seção 4 - Dataframe

Dataframes podem ser acessados também usando a seleção por booleanos:

In [13]:
mais_velhos_que_a_media = cientistas[ cientistas['Age'] > cientistas['Age'].mean() ]
print(mais_velhos_que_a_media)

                   Name        Born        Died  Age     Occupation
1        William Gosset  1876-06-13  1937-10-16   61   Statistician
2  Florence Nightingale  1820-05-12  1910-08-13   90          Nurse
3           Marie Curie  1867-11-07  1934-07-04   66        Chemist
7          Johann Gauss  1777-04-30  1855-02-23   77  Mathematician


A seleção de linhas pode ser feita usando fatiamento no índice:

In [14]:
primeira_metade = cientistas[:4]
segunda_metade = cientistas[4:]

print(primeira_metade)
print('-----')
print(segunda_metade)

                   Name        Born        Died  Age    Occupation
0     Rosaline Franklin  1920-07-25  1958-04-16   37       Chemist
1        William Gosset  1876-06-13  1937-10-16   61  Statistician
2  Florence Nightingale  1820-05-12  1910-08-13   90         Nurse
3           Marie Curie  1867-11-07  1934-07-04   66       Chemist
-----
            Name        Born        Died  Age          Occupation
4  Rachel Carson  1907-05-27  1964-04-14   56           Biologist
5      John Snow  1813-03-15  1858-06-16   45           Physician
6    Alan Turing  1912-06-23  1954-06-07   41  Computer Scientist
7   Johann Gauss  1777-04-30  1855-02-23   77       Mathematician


Broadcasting também se aplica aqui:

In [15]:
print(cientistas*2)

                                       Name                  Born  \
0        Rosaline FranklinRosaline Franklin  1920-07-251920-07-25   
1              William GossetWilliam Gosset  1876-06-131876-06-13   
2  Florence NightingaleFlorence Nightingale  1820-05-121820-05-12   
3                    Marie CurieMarie Curie  1867-11-071867-11-07   
4                Rachel CarsonRachel Carson  1907-05-271907-05-27   
5                        John SnowJohn Snow  1813-03-151813-03-15   
6                    Alan TuringAlan Turing  1912-06-231912-06-23   
7                  Johann GaussJohann Gauss  1777-04-301777-04-30   

                   Died  Age                            Occupation  
0  1958-04-161958-04-16   74                        ChemistChemist  
1  1937-10-161937-10-16  122              StatisticianStatistician  
2  1910-08-131910-08-13  180                            NurseNurse  
3  1934-07-041934-07-04  132                        ChemistChemist  
4  1964-04-141964-04-14  112     

## Seção 5 - Fazendo alterações em Series e em DataFrames

Podemos criar mais colunas atribuindo ao dataframe a nova coluna assim: df['novacoluna'] = dados

Por exemplo, vamos converter as strings "Born" e "Died" para tipos de data e adicionar ao dataframe:

In [16]:
born_datetime = pd.to_datetime(cientistas['Born'], format='%Y-%m-%d')
died_datetime = pd.to_datetime(cientistas['Died'], format='%Y-%m-%d')

cientistas['born_dt'], cientistas['died_dt'] = (born_datetime, died_datetime)

print(cientistas.head())
print('-----')
print('Tipo de dados da coluna Born: ', cientistas['Born'].dtype)
print('Tipo de dados da coluna born_dt: ', cientistas['born_dt'].dtype)
print('-----')
cientistas.info()

                   Name        Born        Died  Age    Occupation    born_dt  \
0     Rosaline Franklin  1920-07-25  1958-04-16   37       Chemist 1920-07-25   
1        William Gosset  1876-06-13  1937-10-16   61  Statistician 1876-06-13   
2  Florence Nightingale  1820-05-12  1910-08-13   90         Nurse 1820-05-12   
3           Marie Curie  1867-11-07  1934-07-04   66       Chemist 1867-11-07   
4         Rachel Carson  1907-05-27  1964-04-14   56     Biologist 1907-05-27   

     died_dt  
0 1958-04-16  
1 1937-10-16  
2 1910-08-13  
3 1934-07-04  
4 1964-04-14  
-----
Tipo de dados da coluna Born:  object
Tipo de dados da coluna born_dt:  datetime64[ns]
-----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Name        8 non-null      object        
 1   Born        8 non-null      object        
 2   Died        8 non-null     

Podemos remover colunas usando o método drop:

In [17]:
# De acordo com a documentação:
# axis: Whether to drop labels from the index (0 or ‘index’) or columns (1 or ‘columns’).
cientistas = cientistas.drop(['Age'], axis=1)
print(cientistas.head())

                   Name        Born        Died    Occupation    born_dt  \
0     Rosaline Franklin  1920-07-25  1958-04-16       Chemist 1920-07-25   
1        William Gosset  1876-06-13  1937-10-16  Statistician 1876-06-13   
2  Florence Nightingale  1820-05-12  1910-08-13         Nurse 1820-05-12   
3           Marie Curie  1867-11-07  1934-07-04       Chemist 1867-11-07   
4         Rachel Carson  1907-05-27  1964-04-14     Biologist 1907-05-27   

     died_dt  
0 1958-04-16  
1 1937-10-16  
2 1910-08-13  
3 1934-07-04  
4 1964-04-14  


## Seção 6 - Exportando e importando dados

Para exportar dados de um dadaframe ou de uma série, podemos usar o método _to_pickle()_, que faz a exportação em formato binário. A leitura desse tipo de dados deve ser feita com o método _pd.read_pickle()_.

Caso a ideia seja salvar/ler um csv, os métodos _to_csv()_ e _pd.read_csv()_ podem ser utilizados.

Uma questão importante é que o rótulo da linha (o índice) é salvo junto. Assim, ao ler um arquivo, esse rótulo irá virar uma nova coluna. Para salvar sem o rótulo da linha podemos passar o parâmetro index=False para os métodos de exportação.

Para exportar dados para o Excel podemos usar as bibliotecas xlwt ou openpyxl e, aí sim, chamar os métodos _to_excel()_ no dataframe/série.

Para exportar dados binários que podem ser lidos no R podemos usar a biblioteca feather e o método _to_feather()_.